How different feature extraction methods affect the effectiveness of classification?

In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import (KFold, LeaveOneOut, RandomizedSearchCV,
                                     RepeatedKFold, ShuffleSplit,
                                     cross_val_score, train_test_split)
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.applications import VGG16, InceptionV3, MobileNetV2
from tensorflow.keras.preprocessing.image import (ImageDataGenerator,
                                                  img_to_array, load_img)

train_dir = Path('./data/train')
train_filepaths = list(train_dir.glob(r'**/*.jpg'))

test_dir = Path('./data/test')
test_filepaths = list(test_dir.glob(r'**/*.jpg'))

val_dir = Path('./data/validation')
val_filepaths = list(val_dir.glob(r'**/*.jpg'))

aug_dir = Path('./data/augmented/')
aug_filepaths = list(aug_dir.glob(r'**/*.jpg')) + list(aug_dir.glob(r'**/*.jpeg'))

data = train_filepaths + test_filepaths + val_filepaths + aug_filepaths

In [ ]:
def paths_to_dataframe(path):
    labels = []
    for i in range(len(path)):
        labels.append(str(path[i]).split(os.sep)[-2])

    labels = pd.Series(labels, name='Label')
    path = pd.Series(path, name='Path').astype(str)

    df = pd.concat([path, labels], axis=1)

    df = df.sample(frac=1).reset_index(drop = True)

    return df
    
data_df = paths_to_dataframe(data)

In [ ]:
data_df = data_df.sample(frac=0.1, random_state=42)
data_df.shape

In [ ]:
# X, X_test, y, y_test = train_test_split(data_df['Path'], data_df['Label'], test_size=0.2, random_state=42)
k = 2
kf = KFold(n_splits=k, shuffle=True, random_state=42)
# kf = KFold(n_splits=2, n_repeats=2, random_state=42)

X = np.array(data_df['Path'])
y = np.array(data_df['Label'])
# X_test = np.array(X)
# y_test = np.array(y)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)

    classification_rep = classification_report(y_test, y_pred)
    print("Classification Report:")
    print(classification_rep)

    classes = np.unique(y_test)
    class_accuracy = {}
    for cls in classes:
        indices = np.where(y_test == cls)[0]
        class_accuracy[cls] = accuracy_score(y_test[indices], np.array(y_pred)[indices])

    return accuracy, precision, recall, f1, class_accuracy

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def nested_dichotomy(X_train, y_train, X_test):
    unique_labels = np.unique(y_train)
    num_classes = len(unique_labels)
    classifiers = []
    classified_indices = np.full(len(X_train), False)

    for i in unique_labels:
        y_binary = np.where(y_train == i, 1, 0)
        tree = DecisionTreeClassifier()
        tree.fit(X_train, y_binary)
        classifiers.append((tree, i))
        print(i)

    predictions = []
    for tree, positive_label in classifiers:
        binary_prediction = tree.predict(X_test)
        predictions.append(np.where(binary_prediction == 1, positive_label, None))

    results = ['tomato' for _ in range(len(X_test))]
    for i in range(len(predictions)):
        for j in range(len(X_test)):
            if predictions[i][j] is not None:
                results[j] = unique_labels[i]
        

    return results, predictions, classifiers

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input


def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.flatten()

In [ ]:
# vgg16_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
# X_train_vgg16 = np.array([extract_features(img_path, vgg16_model) for img_path in X_train])
# X_test_vgg16 = np.array([extract_features(img_path, vgg16_model) for img_path in X_test])
# results_vgg16, pred_vgg16, models_vgg16 = nested_dichotomy(X_train_vgg16, y_train, X_test_vgg16)
# accuracy_vgg16, precision_vgg16, recall_vgg16, f1_vgg16, class_accuracy_vgg16 = metrics(y_test, results_vgg16)

In [ ]:
# from tensorflow.keras.applications import InceptionV3

# inception_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
# X_train_inception = np.array([extract_features(img_path, inception_model) for img_path in X_train])
# X_test_inception = np.array([extract_features(img_path, inception_model) for img_path in X_test])
# results_inception, pred_inception, models_inception = nested_dichotomy(X_train_inception, y_train, X_test_inception)
# accuracy_inception, precision_inception, recall_inception, f1_inception, class_accuracy_inception = metrics(y_test, results_inception)

In [ ]:

def experiment_loop():

    # feature_selection = [VGG16(weights='imagenet', include_top=False, pooling='avg'),
    #                      InceptionV3(weights='imagenet', include_top=False, pooling='avg'),
    #                      MobileNetV2(weights='imagenet', include_top=False, pooling='avg')]
    
    feature_selection = [MobileNetV2(weights='imagenet', include_top=False, pooling='avg')]
        
    scores = [[] for _ in range(k)]

    for fold, (train_index, val_index) in enumerate(kf.split(X)):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        for fs_count, fs in enumerate(feature_selection):
            X_train_fs = np.array([extract_features(img_path, fs) for img_path in X_train])
            X_val_fs = np.array([extract_features(img_path, fs) for img_path in X_val])
            results, pred, models = nested_dichotomy(X_train_fs, y_train, X_val_fs)
            accuracy, precision, recall, f1, class_accuracy = metrics(y_val, results)
            scores[fold].append((accuracy, precision, recall, f1, class_accuracy))

    return scores

scores = experiment_loop()


In [ ]:
from tensorflow.keras.applications import MobileNetV2

mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
X_train_mobilenet = np.array([extract_features(img_path, mobilenet_model) for img_path in X])
X_test_mobilenet = np.array([extract_features(img_path, mobilenet_model) for img_path in X_test])
results_mobilenet, pred_mobilenet, models_mobilenet = nested_dichotomy(X_train_mobilenet, y, X_test_mobilenet)
accuracy_mobilenet, precision_mobilenet, recall_mobilenet, f1_mobilenet, class_accuracy_mobilenet = metrics(y_test, results_mobilenet)

In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.bar(class_accuracy_mobilenet.keys(), class_accuracy_mobilenet.values(), color='skyblue')
plt.title('Accuracy for Each Class (MobileNet)')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.subplot(1, 3, 2)
plt.bar(class_accuracy_inception.keys(), class_accuracy_inception.values(), color='skyblue')
plt.title('Accuracy for Each Class (InceptionV3)')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.subplot(1, 3, 3)
plt.bar(class_accuracy_vgg16.keys(), class_accuracy_vgg16.values(), color='skyblue')
plt.title('Accuracy for Each Class (VGG16)')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.xticks(rotation=90, ha='right')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

methods = ['MobileNet', 'InceptionV3', 'VGG16']
accuracies = [accuracy_mobilenet, accuracy_inception, accuracy_vgg16]
precisions = [precision_mobilenet, precision_inception, precision_vgg16]
recalls = [recall_mobilenet, recall_inception, recall_vgg16]
f1_scores = [f1_mobilenet, f1_inception, f1_vgg16]

plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
plt.bar(methods, accuracies, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Average Accuracy')
plt.xlabel('Feature extraction method')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(2, 2, 2)
plt.bar(methods, precisions, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Average Precision')
plt.xlabel('Feature extraction method')
plt.ylabel('Precision')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(2, 2, 3)
plt.bar(methods, recalls, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Average Recall')
plt.xlabel('Feature extraction method')
plt.ylabel('Recall')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(2, 2, 4)
plt.bar(methods, f1_scores, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Average F1-Score')
plt.xlabel('Feature extraction method')
plt.ylabel('F1-Score')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()